In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool

In [2]:
dataset = pd.read_csv('data/train.csv')

In [3]:
dataset.tail()

,id,race,date,time,circuit,latitude,longitude,altitude,driver,carNumber,constructor,avgDriverFinish,avgConstructorFinish,lapNumber,lapPosition,pitStop,pitCount,pitTime_ms,lapTime_ms
29995,29996,Azerbaijan Grand Prix,2021-06-06,12:00:00,Baku City Circuit,40.3725,49.85330,-7,"Alonso, Fernando",14,Alpine F1 Team,7.448925,5.303571,39,11,0,2,0.0,106314
29996,29997,Chinese Grand Prix,2016-04-17,06:00:00,Shanghai International Circuit,31.3389,121.22000,5,"Rosberg, Nico",6,Mercedes,6.766990,2.199262,53,1,0,2,0.0,100814
29997,29998,European Grand Prix,2002-06-23,NaN,Nürburgring,50.3356,6.94750,578,"Trulli, Jarno",14,Renault,11.580000,5.199507,40,6,0,0,0.0,95675
29998,29999,United States Grand Prix,2014-11-02,20:00:00,Circuit of the Americas,30.1328,-97.64110,161,"Rosberg, Nico",6,Mercedes,6.766990,2.199262,13,1,0,0,0.0,104406
29999,30000,Italian Grand Prix,1997-09-07,NaN,Autodromo Nazionale di Monza,45.6156,9.28111,162,"Schumacher, Michael",5,Ferrari,4.325879,2.833984,16,7,0,0,0.0,87347


In [4]:
dataset.drop(['id', 'race', 'carNumber'], axis=1, inplace=True)

In [5]:
dataset['time'].fillna('NaN', inplace=True)

In [6]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [7]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop("lapTime_ms")
test_labels = test_features.pop("lapTime_ms")

In [8]:
# Define the model
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    loss_function='RMSE',
    random_seed=42,
    cat_features=['date', 'time', 'circuit', 'driver', 'constructor', 'pitStop']
)

# Train the model
model.fit(
    train_features,
    train_labels,
    eval_set=(test_features, test_labels),
    verbose=100,
    early_stopping_rounds=50
)

0:	learn: 61502.2932586	test: 66474.6249575	best: 66474.6249575 (0)	total: 247ms	remaining: 4m 6s
100:	learn: 13255.8864192	test: 19319.8242333	best: 19319.8242333 (100)	total: 6.02s	remaining: 53.6s
200:	learn: 6362.9451158	test: 14753.8650140	best: 14753.8650140 (200)	total: 11.8s	remaining: 46.9s
300:	learn: 5101.6632222	test: 14205.6882013	best: 14205.6882013 (300)	total: 17.6s	remaining: 40.9s
400:	learn: 4551.1862652	test: 14009.4656818	best: 14009.4656818 (400)	total: 23.4s	remaining: 34.9s
500:	learn: 4223.9731617	test: 13925.6248470	best: 13925.6248470 (500)	total: 29.3s	remaining: 29.1s
600:	learn: 3961.3730954	test: 13891.7957321	best: 13891.7773718 (599)	total: 35.1s	remaining: 23.3s
700:	learn: 3761.1889909	test: 13874.8755644	best: 13874.3902854 (699)	total: 41s	remaining: 17.5s
800:	learn: 3576.1852271	test: 13847.9993870	best: 13847.2650493 (795)	total: 46.8s	remaining: 11.6s
900:	learn: 3408.2373954	test: 13829.8867077	best: 13829.6954145 (898)	total: 52.6s	remaining: 